In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from unicodedata import normalize
from nltk.stem import SnowballStemmer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mindlarge-train/entity_embedding.vec
/kaggle/input/mindlarge-train/relation_embedding.vec
/kaggle/input/mindlarge-train/news.tsv
/kaggle/input/mindlarge-train/behaviors.tsv
/kaggle/input/semeval/semeval-2017-train.csv
/kaggle/input/semeval/semeval-2013-dev.csv
/kaggle/input/semeval/semeval-2014-test.csv
/kaggle/input/semeval/semeval-2017-dev.csv
/kaggle/input/semeval/semeval-2013-train-all.csv
/kaggle/input/semeval/semeval-2013-train.csv
/kaggle/input/semeval/semeval-2013-test.csv
/kaggle/input/semeval/semeval-2014-sarcasm.csv
/kaggle/input/semeval/semeval-2017-test.csv


First we get the training dataframe.

In [7]:
df1 = pd.read_csv('/kaggle/input/semeval/semeval-2017-train.csv', sep='\t')
df2 = pd.read_csv('/kaggle/input/semeval/semeval-2013-train-all.csv', sep='\t')

In [45]:
Semeval_train = pd.concat([df1, df2], ignore_index=True)

In [48]:
from datasets import load_dataset
Go_emotions = load_dataset("go_emotions")
Go_emotions_train = Go_emotions['train'].to_pandas()

In [47]:
# Load the news data
mindlarge_train = pd.read_csv('/kaggle/input/mindlarge-train/news.tsv', sep='\t', names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'])

In [9]:
Semeval_train.head()

,label,text
0,1,One Night like In Vegas I make dat Nigga Famous
1,1,Walking through Chelsea at this time of day is...
2,0,"And on the very first play of the night, Aaron..."
3,0,"Drove the bike today, about 40 miles. Felt lik..."
4,-1,looking at the temp outside....hpw did it get ...
...,...,...
107753,1,Today *very* rare day when Democrats will get ...
107754,1,RT @StevenTDennis: Today *very* rare day when ...
107755,-1,@charliemax Democrats will quickly implode and...
107756,-1,Once again Democrats spent all night and this ...


In [49]:
Go_emotions_train.head()

,text,labels,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj


In [43]:
mindlarge_train.head()

,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [13]:
Semeval_train.label.value_counts()

label
 0    48808
 1    40200
-1    18750
Name: count, dtype: int64

In [32]:
Go_emotions_train.text.value_counts()

labels
[27]        12823
[0]          2710
[4]          1873
[15]         1857
[1]          1652
            ...  
[26, 27]        1
[5, 27]         1
[17, 26]        1
[18, 22]        1
[7, 27]         1
Name: count, Length: 7130, dtype: int64

In [46]:
Semeval_train.shape

(107758, 2)

In [35]:
Go_emotions_train.shape

(43410, 3)

In [44]:
mindlarge_train.shape

(101527, 8)

In [ ]:
Semeval_train = pd.read_csv('/kaggle/input/semeval/semeval-2017-train.csv', delimiter='	')
Semeval_train.columns = ['Sentiment', 'Tweet']
Semeval_train.rename(columns={'label': 'Sentiment','text' : 'Tweet'})

Secondly we obtain the prediction dataframe.

In [ ]:
Semeval_test = pd.read_csv('/kaggle/input/semeval/semeval-2017-test.csv', delimiter='	')
Semeval_test.columns = ['Sentiment', 'Tweet']
Semeval_test.rename(columns={'label': 'Sentiment','text' : 'Tweet'})

In [ ]:
# Load the news data
news_df = pd.read_csv('/kaggle/input/mindlarge-train/news.tsv', sep='\t', names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'])

# Load the behaviors data
# behaviors_df = pd.read_csv('/kaggle/input/mindlarge-train/behaviors.tsv', sep='\t', names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

In [ ]:
from datasets import load_dataset
Go_emotions_ds = load_dataset("go_emotions")

In [ ]:
# Convert each split to a DataFrame
Go_emotions_train = Go_emotions_ds['train'].to_pandas()
Go_emotions_validation = Go_emotions_ds['validation'].to_pandas()
Go_emotions_test = Go_emotions_ds['test'].to_pandas()

In [ ]:
# Concatenate all datasets vertically along rows
combined_df = pd.concat([
    news_df, 
    Semeval_train, 
    Semeval_test, 
    Go_emotions_train, 
    Go_emotions_validation, 
    Go_emotions_test
], axis=1, ignore_index=True)

In [ ]:
# Step 1: Determine the minimum length of the DataFrames
min_length = min(len(news_df), len(Semeval_train), 
                 len(Go_emotions_train))

In [ ]:
# Step 2: Truncate each DataFrame to the minimum length
combined_df_truncated = combined_df.iloc[:min_length]

In [ ]:
# Define the new column names
new_column_names = [
    "news_id", "category", "subcategory", "title", "abstract", 
    "url", "title_entities", "abstract_entities", "Sentiment", 
    "Tweet", "Sentiment1", "Tweet1", "text", "labels", "id", 
    "text1", "labels1", "id1",  "text2", "labels2", "id2"
]

# Set the new column names to the combined DataFrame
combined_df_truncated.columns = new_column_names

In [ ]:
combined_df_truncated = combined_df_truncated.drop(columns = ['news_id', 'Sentiment1', 'Tweet1', 'text1', "labels1", "id1",  "text2", "labels2", "id2" ])

In [ ]:
print(combined_df_truncated.head(5))

In [ ]:
print(len(combined_df_truncated.columns))

In [ ]:
print(len(combined_df_truncated))

In [ ]:
print(combined_df_truncated.shape)

In [ ]:
print(combined_df_truncated.isnull().sum())

In [ ]:
combined_df_truncated = combined_df_truncated.dropna(subset=['abstract', 'title_entities', 'abstract_entities'])

In [ ]:
print(combined_df_truncated.shape)

In [ ]:
print(combined_df_truncated.isnull().sum())

In [ ]:
combined_df_truncated

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the text data
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
text_vectorized = vectorizer.fit_transform(combined_df_truncated['text'])

# Fit LDA model
lda = LatentDirichletAllocation(n_components=5, random_state=0)  # Adjust 'n_components' for the number of topics
topics = lda.fit_transform(text_vectorized)

# Assign the topic with the highest probability to each document
combined_df_truncated.loc[:, 'topic'] = topics.argmax(axis=1)


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis
combined_df_truncated.loc[:, 'sentiment 2.0'] = combined_df_truncated['text'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [ ]:
!pip install empath

In [ ]:
from empath import Empath

# Initialize Empath
lexicon = Empath()

# Extract psycholinguistic features (e.g., 'emotional tone' from Empath or a specific feature)
combined_df_truncated.loc[:, 'psycholinguistic'] = combined_df_truncated['text'].apply(lambda x: lexicon.analyze(x, categories=['emotional']))  # Specify categories


In [ ]:
# Save combined DataFrame to a CSV file
combined_df_truncated.to_csv("combined_dataset_with_nlp.csv", index=False)


In [ ]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

In [ ]:
# Load the pretrained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

In [ ]:
# Define a function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

In [ ]:
# Apply sentiment analysis to the 'text' column
combined_df_truncated['predicted_sentiment'] = combined_df_truncated['text'].apply(predict_sentiment)

In [ ]:
# Display the DataFrame with predicted sentiments
print(combined_df_truncated[['text', 'predicted_sentiment']].head())


In [ ]:
from bertopic import BERTopic


In [ ]:
# Initialize BERTopic
topic_model = BERTopic()

In [ ]:
# Fit the model on the 'Tweet' column
topics, _ = topic_model.fit_transform(combined_df_truncated['text'])

# Add the topic labels to the DataFrame
combined_df_truncated['topic model'] = topics

In [ ]:
# Display the DataFrame with topics
print(combined_df_truncated[['text', 'topic model']].head())

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
# Ensure you download the necessary NLTK resources
nltk.download('punkt')

# Sample LIWC-like dictionary (extend as necessary)
liwc_dict = {
    'emotional': ['happy', 'sad', 'anger', 'joy', 'fear'],
    'cognitive': ['think', 'know', 'believe', 'understand', 'learn'],
    'social': ['friend', 'family', 'relationship', 'community', 'team']
}

In [ ]:
# Function to perform LIWC-like analysis
def liwc_analysis(text):
    # Tokenize the input text
    words = word_tokenize(text.lower())
    
    # Initialize counts for each category
    analysis_results = {category: 0 for category in liwc_dict.keys()}
    
    # Count occurrences of words in each category
    for word in words:
        for category, keywords in liwc_dict.items():
            if word in keywords:
                analysis_results[category] += 1

    return analysis_results

In [ ]:
# Apply LIWC analysis to the 'Tweet' column
combined_df_truncated['psycholinguistic_features'] = combined_df_truncated['text'].apply(liwc_analysis)


In [ ]:
# Display the DataFrame with psycholinguistic features
print(combined_df_truncated[['text', 'psycholinguistic_features']].head())

In [ ]:
# Save combined DataFrame to a CSV file
combined_df_truncated.to_csv("implement_with_nlp.csv", index=False)